In [1]:
from roboflow import Roboflow
rf = Roboflow(api_key="ukQ4cG4ARhtXKcDFzcZx")
project = rf.workspace("restofpklot").project("restpklots")
version = project.version(1)
dataset = version.download("coco")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to RestPKLOTs-1 in coco:: 100%|██████████| 8900/8900 [00:00<00:00, 11730.33it/s]


In [4]:
import os

os.getcwd()

'/home/zazza/Documents/BigData/Project/SmartParkingSystem/parking_detection'

In [ ]:
import requests
import zipfile
import io
import boto3
import os

# AWS S3 setup
os.environ["AWS_ACCESS_KEY_ID"] = "insert your key here"
os.environ["AWS_SECRET_ACCESS_KEY"] = "insert your key here"
s3_client = boto3.client("s3")
BUCKET_NAME = "parking-space-management"
FOLDER_NAME = "images/"

# Roboflow dataset URL (ZIP file)
# dataset_url = "https://universe.roboflow.com/ds/gHZhI5DbhQ?key=XBQ56YWyNz"
dataset_url = "https://pub-e8bbdcbe8f6243b2a9933704a9b1d8bc.r2.dev/parking%2Frois_gopro.zip"
BATCH_SIZE = 100  # Number of images to process in a batch

def upload_batch_to_s3(batch):
    """
    Upload a batch of images to S3.
    :param batch: List of tuples [(file_name, file_content), ...]
    """
    for file_name, file_content in batch:
        s3_key = f"{FOLDER_NAME}{file_name}"  # Define the S3 path for the image
        try:
            s3_client.put_object(Bucket=BUCKET_NAME, Key=s3_key, Body=file_content)
            print(f"Uploaded {file_name} to s3://{BUCKET_NAME}/{s3_key}")
        except Exception as e:
            print(f"Error uploading {file_name}: {e}")

# stream the zip file rather than downloading it all at once
response = requests.get(dataset_url, stream=True)
response.raise_for_status()
print("Downloaded the dataset ZIP file")

# we open the zip file in memory so that we can read the contents without saving to disk
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    batch = [] 
    for file_name in z.namelist():
        if file_name.endswith((".jpg", ".jpeg", ".png", ".JPG")):
            with z.open(file_name) as image_file:
                file_content = image_file.read()
                batch.append((file_name, file_content))
            
            if len(batch) == BATCH_SIZE:
                print(f"Uploading batch of {BATCH_SIZE} images to S3...")
                # we only do this once for every batch because it's cheaper than doing it for every image
                upload_batch_to_s3(batch)
                batch = []  # reset the batch

    # upload remaining images
    if batch:
        print(f"Uploading final batch of {len(batch)} images to S3...")
        upload_batch_to_s3(batch)


Downloaded the dataset ZIP file
Adding images/GOPR6733.JPG to batch...
Adding images/GOPR6727.JPG to batch...
Adding images/GOPR6647.JPG to batch...
Adding images/GOPR6653.JPG to batch...
Adding images/GOPR6684.JPG to batch...
Adding images/GOPR6690.JPG to batch...
Adding images/GOPR6691.JPG to batch...
Adding images/GOPR6685.JPG to batch...
Adding images/GOPR6652.JPG to batch...
Adding images/GOPR6646.JPG to batch...
Adding images/GOPR6726.JPG to batch...
Adding images/GOPR6732.JPG to batch...
Adding images/GOPR6718.JPG to batch...
Adding images/GOPR6724.JPG to batch...
Adding images/GOPR6730.JPG to batch...
Adding images/GOPR0037.JPG to batch...
Adding images/GOPR6678.JPG to batch...
Adding images/GOPR6650.JPG to batch...
Adding images/GOPR6644.JPG to batch...
Adding images/GOPR6693.JPG to batch...
Adding images/GOPR6687.JPG to batch...
Adding images/GOPR6686.JPG to batch...
Adding images/GOPR6692.JPG to batch...
Adding images/GOPR6645.JPG to batch...
Adding images/GOPR6651.JPG to ba

In [1]:
import utils.modify_coco as mc

base_path = './RestPKLOTs-1/train/images'
coco_json_path = 'RestPKLOTs-1/train/_annotations.coco.json'

In [2]:
# parking_images = mc.read_image_names_from_directories(base_path)
# mc.update_coco_json_with_parking_field(coco_json_path, parking_images)

parking_counts = mc.count_parking_occurrences(coco_json_path)
print(parking_counts)

{'parking1': 3157, 'parking2': 1175, 'parking3': 1892}


In [2]:
parking_images = mc.read_image_names_from_directories(base_path)
mc.update_coco_json_with_date_captured(coco_json_path, parking_images)

Updated RestPKLOTs-1/train/_annotations.coco.json with date_captured information.
